In [1]:
from scipy import spatial, sparse
from scipy.stats import chi2
from collections import Counter
from num2words import num2words
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.externals import joblib 
from nltk.tokenize import word_tokenize
from sklearn.model_selection import KFold
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances

import os
import imp
import gzip
import copy
import nltk
import pickle
import scipy
import string
import gensim
import operator
import datetime
import multiprocessing
import utils as my_utils

from tqdm import tqdm
import numpy as np
import pandas as pd
#import LDA_ELJST as lda
#import ELJST_script_unigram as lda
#import LJST_script_BTM as lda
import ELJST_script_BTM as lda
import matplotlib.pyplot as plt

/home/william18026/.conda/envs/python3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in tqdm(parse(path)):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [3]:
def process_df(df):
    df['text'] = my_utils.preprocess(df['reviewText'])
#     df['text'] = df['cleaned'].apply(lambda x: " ".join(x))
    return df

In [4]:
dataset_ = getDF('datasets/reviews_Electronics_5.json.gz')
dataset_.shape

1689188it [01:25, 19700.82it/s]


(1689188, 9)

In [16]:
dataset = dataset_.sample(100000)

In [5]:
n_cores = 35

In [6]:
n = int(dataset.shape[0]/n_cores)
list_df = [dataset[i:i+n] for i in range(0, dataset.shape[0],n)]

pool = multiprocessing.Pool(n_cores)
processed_list_df = pool.map(process_df, list_df)
pool.close()

dataset = pd.concat(processed_list_df)
dataset.shape

(1689188, 10)

In [8]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf8')
    model = {}
    for line in tqdm(f):
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

In [ ]:
# %%time
# embedding_dim = 300
# embeddings_index = loadGloveModel("nongit_resources/glove.6B.300d.txt")

In [9]:
n_topics = 5
n_sentiment = 5
alpha = 0.1/n_topics * np.ones(n_topics)
beta = .01
gamma = 10
gamma = [gamma/(n_topics*n_sentiment)]*n_sentiment

maxiter = 10
lambda_param = 1.0

In [10]:
gamma

[0.4, 0.4, 0.4, 0.4, 0.4]

In [11]:
imp.reload(lda)

<module 'ELJST_script_BTM' from '/home/william18026/Embedding-LJST/ELJST_script_BTM.py'>

In [12]:
sampler = lda.SentimentLDAGibbsSampler(n_topics, alpha, beta, gamma, numSentiments=n_sentiment, minlabel = 0, 
                                       maxlabel = 5, SentimentRange = 5, max_df = .5, min_df = 5, 
                                       lambda_param = lambda_param)

In [17]:
%%time
sampler._initialize_(reviews = dataset.text.tolist(), labels = dataset.overall.tolist(), skipgramwindow=5)

CPU times: user 29min 16s, sys: 32 s, total: 29min 48s
Wall time: 29min 41s


In [18]:
sampler.numbigrams

6727654

In [21]:
# %%time
# word_embeddings = []

# for word in tqdm(sampler.words):
#     emb = embeddings_index.get(word, np.array([0]*embedding_dim))
#     word_embeddings.append(emb)

# word_embeddings = np.array(word_embeddings)

In [22]:
# cutoff = .35
# word_similarity = cosine_similarity(word_embeddings)
# word_similarity = word_similarity > cutoff
# word_similarity = word_similarity.astype(int)
# np.fill_diagonal(word_similarity, 0)

In [23]:
# wordOccuranceMatrixBinary = sampler.wordOccuranceMatrix.copy()
# wordOccuranceMatrixBinary[wordOccuranceMatrixBinary > 1] = 1
# wordOccuranceMatrixBinary.shape

In [24]:
# np.sum(wordOccuranceMatrixBinary)

In [ ]:
# Counter(np.array([i[1] for i in nltk.pos_tag(sampler.words)]))

In [25]:
# pp = np.array([i[1] for i in nltk.pos_tag(sampler.words)])
# pp[pp=='JJ'] = 1
# pp[pp=='JJR'] = 1
# pp[pp=='JJS'] = 1
# pp[pp=='NN'] = 1
# pp[pp=='NNS'] = 1
# pp[pp=='NNP'] = 1
# pp[pp=='NNPS'] = 1
# pp[pp!='1'] = 0
# pp = pp.astype(int)

In [26]:
# np.where(pp!=1)[0].shape

In [27]:
# pp.sum()

In [ ]:
# wordOccuranceMatrixBinary[np.where(pp!=1)[0], :] = 0
# wordOccuranceMatrixBinary[:, np.where(pp!=1)[0]] = 0
# np.sum(wordOccuranceMatrixBinary)

In [ ]:
# def get_edges(i):
#     t = np.where(i>0)[0]
#     comb = combinations(t, 2)    
#     embeds = {j:[] for j in t}

#     for p, q in comb:
#         if word_similarity[p][q]:
#             embeds[p] += [q]
#             embeds[q] += [p]
#     return embeds

In [ ]:
# def get_edges(i):
#     t = np.where(i==0)[0]
#     k = word_similarity.copy()

#     k[t, :] = 0
#     k[:, t] = 0

#     z, x = np.where(k==1)
#     embeds = {}
#     for p, q in zip(z, x):
#         try:
#             embeds[p] += [q]
#         except:
#             embeds[p] = [q]
#         try:
#             embeds[q] += [p]
#         except:
#             embeds[q] = [p]
#     for i in embeds.keys():
#         embeds[i] = list(set(embeds[i]))
#     return embeds

In [ ]:
# %%time
# pool = multiprocessing.Pool(n_cores)
# similar_words = pool.map(get_edges, wordOccuranceMatrixBinary)
# pool.close()

# pickle_out = open("resources/amazon_muiscal_glove_0.4.pickle","wb")
# pickle.dump(similar_words, pickle_out)
# pickle_out.close()

In [29]:
similar_words = pickle.load(open("resources/amazon_electronics_glove_0.35_POS.pickle","rb"))

In [ ]:
sampler.run(reviews=dataset.text.tolist(), labels=dataset.overall.tolist(), 
            similar_words=similar_words, mrf=False, maxIters=maxiter)

Starting iteration 1 of 10


Starting iteration 2 of 10


Starting iteration 3 of 10


Starting iteration 4 of 10


Starting iteration 5 of 10


Starting iteration 6 of 10


Starting iteration 7 of 10


Starting iteration 8 of 10


Starting iteration 9 of 10


Starting iteration 10 of 10


In [ ]:
plt.plot(sampler.loglikelihood_history)

In [ ]:
silhouette_score(euclidean_distances(sampler.wordOccuranceMatrix),
                 sampler.dt_distribution.argmax(axis=1), metric='precomputed')

In [ ]:
davies_bouldin_score(sampler.wordOccuranceMatrix, sampler.dt_distribution.argmax(axis=1))

In [ ]:
my_utils.coherence_score(sampler.wordOccuranceMatrix, list(sampler.getTopKWords(5).values()), sampler.vocabulary)

In [ ]:
%%time
my_utils.get_hscore_multi(sampler.dt_distribution, sampler.wordOccuranceMatrix, n_topics, 3000)

In [ ]:
sampler.loglikelihood()

In [ ]:
sampler.perplexity()

## Appendix

In [ ]:
# joblib.dump(sampler, "resources/BTM_sampler_0.4_1")